In [5]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np 
import time
from threading import Thread
from bqplot import pyplot as plt

from pynq import PL
from pynq import allocate
from pynq import Overlay  #import the overlay module

In [6]:
PL.reset() #important fixes caching issues which have popped up.
ol = Overlay('./design_1.bit')  #locate/point to the bit file
dma = ol.axi_dma_0 #might need to change name depending on what you called it

In [7]:
n = 8192
sample_rate = 4.9152
x_data = [ix/sample_rate for ix in range(n)]
x_data = x_data[::8]
fig = plt.figure(title='Live Data from FPGA', animation_duration=0)
line = plt.plot([], [])  # Initial empty plot

plt.ylim(-250, 250)
plt.ylabel('Voltage')
plt.xlim(0, 1250)  # Initial X-axis range, will update dynamically
plt.xlabel("Time [ns]")

out_buffer = allocate(n, dtype=np.int32)


# Function to update the plot with new sensor data
def update_plot():
    while True:
        if is_running.value:
            q = time.time()
            dma.recvchannel.transfer(out_buffer)
            start_time = time.time()
            dma.recvchannel.wait()
            end_time = time.time()
            line.x = x_data
            line.y = np.float64(out_buffer[::8])
            print(time.time() - q)
            if end_time - start_time > 5:
                print(end_time - start_time)
                time.sleep(3) 


# Toggle button to start/stop the plot
is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(is_running, fig)

# Function to start the thread for continuous plotting
def start_plot(change):
    if is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        thread = Thread(target=update_plot, daemon=True)
        thread.start()

# Watch the button and start the plot when pressed
is_running.observe(start_plot, names='value')

ToggleButton(value=False, description='Running', icon='play', tooltip='Start/Stop the live plot')

Figure(axes=[Axis(label='Time [ns]', scale=LinearScale(max=1250.0, min=0.0)), Axis(label='Voltage', orientatio…

0.007668733596801758
0.02294158935546875


/usr/local/share/pynq-venv/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(
Exception in thread Thread-8 (update_plot):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1621/2269307972.py", line 21, in update_plot
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/dma.py", line 138, in transfer
    raise RuntimeError("DMA channel not idle")
RuntimeError: DMA channel not idle


0.023363590240478516
0.026813030242919922
0.023705482482910156
0.026360511779785156
0.021771669387817383
0.025354623794555664
0.021113157272338867
0.02486443519592285
0.025091171264648438
0.023690223693847656
0.02507305145263672
0.023688554763793945
0.023981809616088867
0.02487468719482422
0.023966312408447266
0.024847984313964844
0.02397465705871582
0.026232481002807617
0.022415876388549805
0.026225566864013672
0.02337336540222168
0.023923397064208984
0.02486443519592285
0.024057865142822266
0.03768014907836914
0.012841224670410156
0.022949695587158203
0.03903388977050781
0.009393930435180664
0.025077104568481445
0.023479461669921875
0.025005102157592773
0.024957656860351562
0.0338587760925293
0.015002727508544922
0.02521514892578125
0.03836417198181152
0.009586334228515625
0.023421049118041992
0.044287681579589844
0.0058155059814453125
0.024823427200317383
0.02325749397277832
0.03686857223510742
0.03977370262145996
0.009630203247070312
0.05813860893249512
0.0056743621826171875
0.0244